# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset
from utils import get_demonstration_fp

In [3]:
import numpy as np
import random
from faker import Faker

# Set a random seed so that the results are reproducible.
Faker.seed(12374098)
random.seed(1928374)
np.random.seed(28374812)

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [4]:
demonstration_fp = get_demonstration_fp()
df = pd.read_csv(demonstration_fp)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1937-10-28,15:53:04,2022-08-05 04:43:34,NaN
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,NaN,12:26:00,2022-08-07 01:56:33,NaN
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1931-09-24,16:08:25,2022-08-04 20:27:37,NaN
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1936-11-30,NaN,2022-08-07 07:05:55,NaN
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1918-11-07,10:59:08,2022-08-02 15:13:34,NaN


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [5]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv(demonstration_fp, dtype=dtypes, parse_dates=["Married since"])  # Parse datetimes with parse_dates
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1937-10-28,15:53:04,2022-08-05 04:43:34,NaN
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,NaN,12:26:00,2022-08-07 01:56:33,NaN
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1931-09-24,16:08:25,2022-08-04 20:27:37,NaN
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1936-11-30,NaN,2022-08-07 07:05:55,NaN
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1918-11-07,10:59:08,2022-08-02 15:13:34,NaN


### Dates and times

While pandas can easily parse datetime columns, it cannot parse proper times and dates. Instead, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

Note the check for NA dates/times, otherwise it will raise an error instead.

In [6]:
df["Birthday"] = [dt.date.fromisoformat(x) if not pd.isna(x) else pd.NA for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) if not pd.isna(x) else pd.NA for x in df["Board time"]]

Now, let's check the data types of our DataFrame:

In [7]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
all_NA                  float64
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [8]:
df["Birthday"][0]

datetime.date(1937, 10, 28)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [9]:
# NBVAL_IGNORE_OUTPUT

meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:127: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [10]:
print(meta_dataset)

# Rows: 891
# Columns: 12

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameter

Note that the column with all NA's has been converted to an exponential distribution. This has no effect on the outcome, since it will generate NA's exclusively.

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [11]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [12]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,228,"Q =2Q2']JOl""1GH 8 gk9kKnA!(X6]p;$e""6;X[#2|]",female,79,4,114.964637,ETVYc!xn^,S,1924-12-04,17:58:03,2022-07-17 13:52:04,NaN
1,675,e;TK3LB@Hej@_C:ocyChbXdX'_{l 5S,female,69,0,0.973345,<NA>,S,None,10:42:55,2022-08-07 08:59:24,NaN
2,788,"75Edxp))Qq#&Wxtu*4e R6?2N7>cxj} [,$xW/RL( Ox i...",male,<NA>,0,73.326058,<NA>,S,1931-10-26,15:11:56,NaT,NaN
3,589,+nu}Djl~Fn1`S1Q\LpO'y:~jbUkA9,female,43,0,51.588363,<NA>,S,1911-09-23,13:00:37,2022-07-20 15:18:30,NaN
4,443,"AK3y=p\SJrMIigt,^P_DkI!B}6F|sBDr}*(=f}FtD|#)c ...",female,43,0,14.081128,<NA>,C,1911-03-27,13:26:47,2022-08-09 14:03:50,NaN


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [13]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [14]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,"""&{V 0x`~I/Q;\36mX9/ttay%,z{jVI] 5 aJQ b!L*>o...",male,50,0,20.361694,<NA>,S,1919-02-20,None,2022-08-03 02:25:11,NaN
1,2,(|s:.t?i(fq3a,male,36,0,23.818851,DW.? F/xt,S,1918-06-10,16:45:25,2022-07-22 09:13:34,NaN
2,3,O7o0Jn1t?/!s^[2m;'%9yL2GQE =S2&vZW:8)bR|7AT;1R...,male,15,1,8.952692,<NA>,S,1918-02-24,17:14:38,2022-07-20 03:56:52,NaN
3,4,nl=Cg-= Z;.\y gB7H*E4 tfY,male,77,2,38.117718,"E:Onprd"")^/2B",S,1919-01-29,13:49:58,2022-07-26 19:17:15,NaN
4,5,"""%#[,v7(%yp[#I4,d"". #Yz|) KN@SYZkdrJt,Q2~@ tO~e",male,18,0,21.840967,<NA>,S,1905-07-13,18:16:09,2022-07-25 05:58:25,NaN


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [15]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Eric Nash,male,0,0,4.673285,<NA>,Q,1911-10-21,16:30:43,2022-07-31 20:14:38,NaN
1,2,Blake Mckay,female,71,0,1.910530,<NA>,Q,1922-04-27,16:43:32,2022-07-21 05:09:24,NaN
2,3,David Newton,male,50,0,3.346805,<NA>,Q,1910-10-17,None,2022-07-23 14:00:27,NaN
3,4,Sara Fernandez,male,54,0,12.172444,<NA>,C,1936-02-08,12:52:50,2022-08-11 00:22:04,NaN
4,5,Edward Roberts,female,49,1,27.814973,Gm~ la,S,None,13:56:13,2022-08-12 02:04:14,NaN


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [16]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Steven Hall,female,22,2,2.205320,C6cM@B4%^1HVD,S,1925-09-17,14:04:58,2022-07-26 23:37:32,NaN
1,2,Jenna Nelson,female,30,0,1.179053,<NA>,S,1931-11-22,18:01:32,2022-07-29 13:22:13,NaN
2,3,Zachary Jones,male,23,0,1.447257,<NA>,S,1914-05-25,14:06:49,2022-07-18 00:14:43,NaN
3,4,Richard Parker,male,<NA>,0,0.875891,<NA>,Q,1930-08-27,13:42:20,2022-07-15 16:38:59,NaN
4,5,Cody Williams,male,27,0,0.377074,<NA>,C,1911-02-12,15:29:09,2022-07-21 21:49:28,NaN


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [17]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name", "prop_missing": 0.1},  # Manually set the proportion of missing values.
    "Fare": {"distribution": "LogNormalDistribution"},  # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)},  # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
0,1,Jennifer Howard,male,25,0,0.506016,<NA>,S,None,14:41:10,2022-08-02 23:03:22,NaN
1,2,Emily Robbins,male,35,0,0.969580,<NA>,S,1929-04-30,16:37:46,NaT,NaN
2,3,Melissa Pierce,male,38,0,2.867970,B67,S,1933-02-14,11:38:30,2022-08-13 08:46:24,NaN
3,4,Tina Myers,female,29,0,0.316032,<NA>,S,1917-04-29,10:52:45,2022-07-20 02:25:39,NaN
4,5,Hailey Nicholson,female,30,0,4.973877,<NA>,S,1931-07-30,11:17:39,2022-07-20 00:22:18,NaN
